In [2]:
import uuid
from flask import Flask, request, jsonify
from utilities import logger
from flask_cors import CORS
import sys
from xinference.client import Client

mylogger = logger.Logger(name='llmtool', debug=True).logger  
# app = Flask(__name__)  
# CORS(app)

# 使用字典模拟用户存储，实际项目中应使用数据库
users_db = {
    "admin": {"password": "123456"},
    'zz': {'password': 'zz'},
    'xyz': {'password': 'abc'}
}

# 存储登录成功的用户session数据
sessions = {}



In [ ]:
x_addr = 'http://10.101.9.50:9998'
mylogger.debug(f"Start to connect xinference server at {x_addr}")
try:
    client = Client(x_addr)    
    model_uid = client.launch_model(model_name="bge-large-zh-v1.5", model_type="embedding")    
    model = client.get_model(model_uid)
except Exception as e:
    mylogger.error(f"Something goes wrong: {e}")   
def create_embedding_bge(sentence):
    result = model.create_embedding(sentence)
    embedding = result['data'][0]['embedding']
    # LOG.debug(f"length of embedding {len(embedding)}, {embedding[:5]}")
    return embedding

In [2]:
vec = create_embedding_bge('test sentence')

In [7]:
import requests

# 调用生成列表的服务
question = '可以根据需要修改question参数'  # 可以根据需要修改question参数
response = requests.post('http://127.0.0.1:5111/genvec', json={'question': question})
vec = create_embedding_bge(question)
if response.status_code == 200:
    float_list = response.json()
    # 假设 float_list 和 vec 是之前获取的列表
# 检查类型是否都是 list
    if isinstance(float_list, list) and isinstance(vec, list):
        # 检查长度是否相同
        if len(float_list) == len(vec):
            # 逐个比较元素是否一致
            if all(a == b for a, b in zip(float_list, vec)):
                mylogger.debug("float_list 和 vec 相同")
            else:
                mylogger.error("float_list 和 vec 不同")
        else:
            mylogger.error("float_list 和 vec 长度不同")
    else:
        mylogger.error("float_list 和 vec 不是列表类型")
        # 判断列表长度
        mylogger.debug(f"Length of the list: {len(float_list)}")    
    
else:
    mylogger.error("Failed to retrieve the list.")

2024-03-22 23:14:21.290 | DEBUG    | __main__:<module>:16 - float_list 和 vec 相同


In [3]:
import mysql.connector
mysql_db_config = {
    "host": "10.101.9.50",
    "user": 'root',
    "password": 'newpass',
    "database": 'codebase'
}

db_connection = mysql.connector.connect(**mysql_db_config)
db_cursor = db_connection.cursor()
if db_connection.is_connected():
    mylogger.debug("Successfully connect mysql database")

mylogger.debug(mysql_db_config)
def execute_query(query, params=()):
    db_cursor.execute(query, params)
    if query.strip().startswith("SELECT"):
        pass
    else:
        db_connection.commit() 

2024-03-23 00:17:17.569 | DEBUG    | __main__:<module>:12 - Successfully connect mysql database
2024-03-23 00:17:17.571 | DEBUG    | __main__:<module>:14 - {'host': '10.101.9.50', 'user': 'root', 'password': 'newpass', 'database': 'codebase'}


In [4]:
query = 'SELECT username, last_refresh FROM user_sessions WHERE userKey = %s'
execute_query(query, ('f3082fc9-bc97-4d91-b600-b860e7cf609c',))
res = db_cursor.fetchone()
if res:
    print(1)
else:
    print(0)

1


In [1]:
! pip install ldap3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
                                              0.0/432.2 kB ? eta -:--:--
     ------------------------------------- 432.2/432.2 kB 13.6 MB/s eta 0:00:00


In [5]:
from ldap3 import Server, Connection, ALL
from ldap3.core.exceptions import LDAPBindError 
LDAP_SERVER = 'ldap://192.168.0.58:389'
LDAP_SEARCH_BASE = 'ou=NSTC,dc=ninestar'

def authenticate(username, password):
    server = Server(LDAP_SERVER, get_info=ALL)
    user_dn = f"cn={username},{LDAP_SEARCH_BASE}"
    try:
        with Connection(server, user=user_dn, password=password, auto_bind=True):
            return True
    except LDAPBindError:
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    
# Define the LDAP server
server = Server(LDAP_SERVER, get_info=ALL)
username = 'zhouzhen'
password = 'Nstc0727'
# username = '10572'
# password = 'Nstc0612'


if authenticate(username, password):
    print(f'Authentication successful, username {username}')
else:
    print(f'Authentication failed, username {username}')



Authentication failed, username zhouzhen


In [7]:
import faiss
index = faiss.read_index('story.faiss')
print(index.ntotal)

33195


In [9]:
import random

random_numbers = [random.randint(0, 33194) for _ in range(1000)]

In [10]:
import numpy as np
# Create a new Faiss index and a mapping dictionary
new_index = faiss.IndexFlatL2(index.d)
mapping = {}

# Iterate over vectors in the index
for i in range(index.ntotal):
    vector = index.reconstruct(i)
    if i not in random_numbers:
        new_index.add(np.array([vector]))
        mapping[i] = new_index.ntotal - 1
    else:
        mapping[i] = -1

# Save the new index
faiss.write_index(new_index, 'new_index.faiss')

# Print the mapping dictionary
print(mapping)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: -1, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: -1, 86: 84, 87: 85, 88: 86, 89: 87, 90: 88, 91: 89, 92: 90, 93: -1, 94: 91, 95: 92, 96: 93, 97: 94, 98: 95, 99: -1, 100: 96, 101: 97, 102: 98, 103: 99, 104: 100, 105: -1, 106: 101, 107: 102, 108: 103, 109: 104, 110: 105, 111: 106, 112: 107, 113: 108, 114: 109, 115: 110, 116: 111, 117: 112, 118: 113, 119: 114, 120: 115, 121: 116, 122: